# Memory

- Memory is an integral part for an Agent - it enables an Agent to learn from previous experiences

- Here, we implement our own memory class in order to store memory in a suitable abstraction space from any modality - text / image / video / audio into memory chunks or graphs, and extract out the relevant chunks or graph components based on the task

- All Memory classes must implement MemoryTemplate, which contains the following core functions:
    - `append`: Takes in a list of memories, processes them and adds it to memory chunk / graph
    - `remove`: Takes in an existing memory and removes it
    - `reset`: Removes all memories
    - `retrieve`: Returns the top memories according to task
    
- Memory class can be used separately as part of a retrieval and storage pipeline that gets interfaced with the Agent via `Global Context` or directly into the task.

- Memory can also be passed into the `Agent` class as part of the `memory_bank`. If done this way, Agents will automatically use the `retrieve` function each time a subtask is done to append the relevant memories into the agent prompt. Note all other functions need to be done separately outside the `Agent` class, so it is best done within Agent wrappers

```python
class MemoryTemplate(ABC):
    """A generic template provided for all memories"""

    @abstractmethod
    def append(self, memory_list, mapper=None):
        """Appends multiple new memories"""
        pass

    @abstractmethod
    def remove(self, existing_memory):
        """Removes an existing_memory. existing_memory can be str, or triplet if it is a Knowledge Graph"""
        pass

    @abstractmethod
    def reset(self):
        """Clears all memories"""

    @abstractmethod
    def retrieve(self, task: str):
        """Retrieves some memories according to task"""
        pass
```



## Some Initial Imports

In [1]:
from taskgen import *

In [2]:
import os
# Important: Make sure you do not commit your own API key
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

## Define your Memory Class Here
- Implement your Memory that overwrites all the required abstracted functions above
- The default Memory Class will be for a Sync Agent

In [4]:
class ListMemory(MemoryTemplate):
    ''' Memory in the form of a list, retrieves using OpenAI embeddings '''
    def __init__(self, memory_list: list = None, top_k: int = 3, database = None):
        ''' Inputs:
        memory_list: list of initial memories
        top_k: Number of memories to retrieve
        database: the mapping of memory to their embeddings
        '''
        self.top_k = top_k
            
        if memory_list is None:
            self.memory_list = []
        else:
            self.memory_list = memory_list
            
        if database is None:
            self.database = {}
        else:
            self.database = database

    def append(self, memory_list, mapper=None):
        """Adds a list of memories"""
        if not isinstance(memory_list, list):
            memory_list = [memory_list]
        self.memory_list.extend(memory_list)

    def remove(self, memory_to_remove):
        """Removes a memory"""
        self.memory.remove(memory_to_remove)

    def reset(self):
        """Clears all memory"""
        self.memory_list = []
        
    def retrieve(self, task: str) -> list:
        """Performs retrieval of top_k similar memories according to embedding similarity"""
        import heapq
        import numpy as np
        task_embedding = self.get_or_create_embedding(task)
        memory_score = [np.dot(self.get_or_create_embedding(memory), task_embedding) for memory in self.memory_list]
        top_k_indices = heapq.nlargest(self.top_k, range(len(self.memory_list)), key=lambda i: memory_score[i])
        return [self.memory_list[index] for index in top_k_indices]
        
    def get_or_create_embedding(self, text):
        if text in self.database:
            return self.database[text]
        else:
            from openai import OpenAI
            client = OpenAI()
            cleaned_text = text.replace("\n", " ")
            embedding = client.embeddings.create(input=[cleaned_text], model="text-embedding-3-small").data[0].embedding
            self.database[text] = embedding
            return embedding

### Use Memory without the Agent

In [5]:
memory = ListMemory(['hello', 'goodbye', 'good night', 'good morning'], top_k = 1)

In [6]:
memory.retrieve('good day')

['good morning']

### Use Memory in an Agent's Memory Bank

In [7]:
def greet_user(greeting: str) -> str:
    ''' Outputs greeting to the user '''
    return greeting

In [8]:
agent = Agent('Greeting Generator', 'Greets user with the greeting present in Greet_Memory', 
              llm = llm).assign_functions([greet_user])
agent.memory_bank['Greet_Memory'] = ListMemory(['hello', 'goodbye', 'good night', 'good morning'], top_k = 1)

In [9]:
agent.run('It is a bright sunny day')

Observation: No subtasks have been completed yet for the assigned task, which is to greet the user based on the context of a bright sunny day.
Thoughts: To complete the assigned task, I can use the greeting stored in Greet_Memory, which is "good morning", as it is appropriate for a bright sunny day.
Subtask identified: Use the greet_user function to output the greeting "good morning" to the user.
Calling function greet_user with parameters {'greeting': 'good morning'}
> {'output_1': 'good morning'}

Observation: The greeting "good morning" has been successfully generated and outputted to the user.
Thoughts: Since the greeting has been provided, the task appears to be complete. There is no further action required as the user has been greeted appropriately for the bright sunny day.
Subtask identified: End Task
Task completed successfully!



[{'output_1': 'good morning'}]

# Async Memory
- Async memory is possible. Async memory must have async for the functions involving async processes
- Start your Async memory class name with Async

- **NOTE: When you contribute memory classes, it is not necessary to contribute the Async version**
- **This is only for more production-based use cases**

In [10]:
async def llm_async(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import AsyncOpenAI
    
    # define your own LLM here
    client = AsyncOpenAI()
    response = await client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [11]:
class AsyncListMemory(MemoryTemplate):
    ''' Memory in the form of a list, retrieves using OpenAI embeddings '''
    def __init__(self, memory_list: list = None, top_k: int = 3, database = None):
        ''' Inputs:
        memory_list: list of initial memories
        top_k: Number of memories to retrieve
        database: the mapping of memory to their embeddings
        '''
        self.top_k = top_k
            
        if memory_list is None:
            self.memory_list = []
        else:
            self.memory_list = memory_list
            
        if database is None:
            self.database = {}
        else:
            self.database = database

    def append(self, memory_list, mapper=None):
        """Adds a list of memories"""
        if not isinstance(memory_list, list):
            memory_list = [memory_list]
        self.memory_list.extend(memory_list)

    def remove(self, memory_to_remove):
        """Removes a memory"""
        self.memory.remove(memory_to_remove)

    def reset(self):
        """Clears all memory"""
        self.memory_list = []
        
    async def retrieve(self, task: str) -> list:
        """Performs retrieval of top_k similar memories according to embedding similarity"""
        import heapq
        import numpy as np
        import asyncio
        task_embedding = await self.get_or_create_embedding(task)
        # Await the coroutines to get the embeddings first
        memory_embeddings = await asyncio.gather(*[self.get_or_create_embedding(memory) for memory in self.memory_list])
        # Now, perform the dot product between each memory embedding and task_embedding
        memory_score = [np.dot(memory_embedding, task_embedding) for memory_embedding in memory_embeddings]
        top_k_indices = heapq.nlargest(self.top_k, range(len(self.memory_list)), key=lambda i: memory_score[i])
        return [self.memory_list[index] for index in top_k_indices]
        
    async def get_or_create_embedding(self, text):
        if text in self.database:
            return self.database[text]
        else:
            from openai import AsyncOpenAI
            client = AsyncOpenAI()
            cleaned_text = text.replace("\n", " ")
            response = await client.embeddings.create(input=[cleaned_text], model="text-embedding-3-small")
            embedding = response.data[0].embedding
            self.database[text] = embedding
            return embedding

### Use Memory without the Agent

In [12]:
memory = AsyncListMemory(['hello', 'goodbye', 'good night', 'good morning'], top_k = 1)

In [13]:
await memory.retrieve('good day')

['good morning']

### Use Memory in an Agent's Memory Bank

In [14]:
def greet_user(greeting: str) -> str:
    ''' Outputs greeting to the user '''
    return greeting

In [15]:
agent = AsyncAgent('Greeting Generator', 'Greets user with the greeting present in Greet_Memory', 
              llm = llm_async).assign_functions([greet_user])
agent.memory_bank['Greet_Memory'] = AsyncListMemory(['hello', 'goodbye', 'good night', 'good morning'], top_k = 1)

In [16]:
await agent.run('It is a bright sunny day')

Observation: No subtasks have been completed yet for the assigned task, which is to greet the user based on the context of a bright sunny day.
Thoughts: To complete the assigned task, I can use the greeting stored in Greet_Memory, which is "good morning", to greet the user appropriately.
Subtask identified: Use the greet_user function to output the greeting "good morning" to the user.
Calling function greet_user with parameters {'greeting': 'good morning'}
> {'output_1': 'good morning'}

Observation: The greeting "good morning" has been successfully generated and outputted to the user.
Thoughts: Since the greeting has been provided, the next step is to determine if any further interaction or output is needed based on the context of a bright sunny day.
Subtask identified: End Task
Task completed successfully!



[{'output_1': 'good morning'}]